#### 

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, FloatType, TimestampType
from pyspark.sql.functions import col, array_contains, isnan, when, count
from pyspark.sql.functions import lit, concat_ws, concat, collect_list, udf
from pyspark.sql.functions import countDistinct

spark = SparkSession.builder.appName("answers").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
path = "D:\Big_data\customer_analysis\archive2019-Nov.csv"
df = spark.read.option("header",'True').option('delimiter', ',').csv(path)
df.printSchema()

In [ ]:
schema = StructType() \
      .add("event_time",TimestampType(),True) \
      .add("event_type",StringType(),True) \
      .add("product_id",StringType(),True) \
      .add("category_id",StringType(),True) \
      .add("category_code",StringType(),True) \
      .add("brand",StringType(),True) \
      .add("price",DoubleType(),True) \
      .add("user_id",StringType(),True) \
      .add("user_session",StringType(),True) \
      
      
    
      

df = spark.read.format("csv") \
      .option("header", True) \
      .option('delimiter', ',') \
      .schema(schema) \
      .load("/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv")

df.printSchema()

In [ ]:
df.show()

## Question 1

In [ ]:
col=df.select('event_type').distinct()

In [ ]:
col.show()

In [ ]:
ex_df=df.groupBy('category_id','event_type').agg(count('*').alias('counts'))

In [ ]:
ex_df.show()

In [ ]:
f_df=ex_df.groupBy("category_id").pivot("event_type", ["view", "cart"]).sum("counts")

In [ ]:
import pyspark.sql.functions as F
f_df.withColumn("cart2viewtratio", (F.col('cart') / F.col('view'))).orderBy('cart2viewtratio',ascending=False).show()

## question 2

In [ ]:
w_df=df.groupBy('event_time','event_type','brand').agg(count('*').alias('counts'))

In [ ]:
w_df.show()

In [ ]:
from pyspark.sql.functions import hour , minute , second
df2 = w_df.filter(((hour(df.event_time) == 6) & (minute(df.event_time) == 0) & (second(df.event_time) == 0)) | (hour(df.event_time) <= 5))

In [ ]:
df3=df2.groupBy("Brand").pivot("event_type", ["view", "purchase"]).sum("counts")
df3.show()

In [ ]:
df3=df3.withColumn("purchase2viewratio", (F.col('purchase') / F.col('view')))

In [ ]:
x=df3.orderBy('purchase2viewratio',ascending=False)
x.show()


In [ ]:
y=x.first()
print(y[0])

### brand found is :pixiebelles



## question 3

In [ ]:
w_df1=df.groupBy('event_time','event_type','brand','category_id').agg(count('*').alias('counts'))

In [ ]:
from pyspark.sql.functions import hour , minute , second
df32 = w_df1.filter(((hour(df.event_time) == 6) & (minute(df.event_time) == 0) & (second(df.event_time) == 0)) | (hour(df.event_time) <= 5))

In [ ]:

df33=df32.groupBy("Brand",'category_id').pivot("event_type", ["view", "purchase"]).sum("counts")
df33=df33.withColumn("purchase2viewratio", (F.col('purchase') / F.col('view')))
df33.where( (df33.Brand=='pixiebelles')).orderBy('purchase2viewratio',ascending=False).show()

## question 4

In [ ]:
from pyspark.sql.functions import sum as _sum
df44=df.groupBy('category_id','event_type').agg(_sum('price').alias('revenue'))
df44=df44.where(df44.event_type=='purchase')
df44.orderBy('revenue',ascending=False).show()